In [31]:
import numpy as np

In [32]:
########## what if there are no unknowns
########## what if no .circuit or no .end
########## check for det = 0 before np.linalg.solve
########## figure other possible edge cases
########## R = 0 cases

In [33]:
fptr = open('testdata/test_1.ckt', 'r')

lines = fptr.readlines()
lines = [line.strip("\n").strip() for line in lines]

In [34]:
circuit_flag = False
circuit = []

for line in lines:
    if line == '.end':
        break
 
    if circuit_flag == True:
        circuit.append(line.split())
    
    if line == '.circuit':
        circuit_flag = True

circuit

[['V1', '1', 'GND', 'dc', '2'], ['R1', '1', '2', '1'], ['R2', '2', 'GND', '1']]

In [35]:
elements = []
for element in circuit:
    dict = {}
    dict['name'] = element[0]
    dict['type'] = element[0][0]
    dict['nodes'] = (element[1], element[2])         # 1st is +ve and 2nd is -ve terminal
    dict['value'] = float(element[-1])
    elements.append(dict)
        
elements

[{'name': 'V1', 'type': 'V', 'nodes': ('1', 'GND'), 'value': 2.0},
 {'name': 'R1', 'type': 'R', 'nodes': ('1', '2'), 'value': 1.0},
 {'name': 'R2', 'type': 'R', 'nodes': ('2', 'GND'), 'value': 1.0}]

In [36]:
n = [] ############## Not taking GND here
for element in elements:
    [n.append(node) for node in element['nodes'] if node not in n and node != 'GND']
n

['1', '2']

In [37]:
I_elements = [element for element in elements if element['type'] == 'I']
V_elements = [element for element in elements if element['type'] == 'V']

R_count = sum(1 for element in elements if element['type'] == 'R')
V_count = len(V_elements)
I_count = len(I_elements)
V_elements

[{'name': 'V1', 'type': 'V', 'nodes': ('1', 'GND'), 'value': 2.0}]

In [41]:
A = np.full([len(n)+V_count, len(n)+V_count], 0)           # Default value of every entry is 0

for i in range(len(n)):
    for j in range(i, len(n)):
        G = 0
        for element in elements:
            if n[i] in element['nodes'] and n[j] in element['nodes'] and element['type'] == 'R':
                G += 1/element['value']
        if i == j:
            A[i][j] = G
        else:
            A[i][j] = -G


for i in range(V_count):
    for j in range(len(n)):
        element = V_elements[i]
        if n[j] in element['nodes'] and element['type'] == 'V':
            if n[j] == element['nodes'][0]:
                A[j][i+len(n)] = 1
            elif n[j] == element['nodes'][1]:
                A[j][i+len(n)] = -1


for i in range(len(n)+V_count):
    for j in range(i, len(n)+V_count):
        A[j, i] = A[i, j]
    
A

array([[ 1, -1,  1],
       [-1,  2,  0],
       [ 1,  0,  0]])

In [44]:
b = np.full(len(n)+V_count, 0)


for i in range(len(n)):
    for element in I_elements:
        if n[i] == element['nodes'][0]:
            b[i] += element['value']
        elif n[i] == element['nodes'][1]:
            b[i] -= element['value']

for i in range(V_count):
    for element in V_elements:
        if n[i] == element['nodes'][0]:
            b[i+len(n)] += element['value']
        elif n[i] == element['nodes'][1]:
            b[i+len(n)] -= element['value']

b

array([0, 0, 2])

In [45]:
x = np.linalg.solve(A, b)
x

array([ 2.,  1., -1.])

In [47]:
V_elements

[{'name': 'V1', 'type': 'V', 'nodes': ('1', 'GND'), 'value': 2.0}]

In [51]:
Voltages = {}
Currents = {}
Voltages['GND'] = 0
for i in range(len(n)):
    Voltages[n[i]] = x[i]
for i, V_element in enumerate(V_elements):
    Currents[V_element['name']] = x[i+len(n)]

Voltages, Currents

({'GND': 0, '1': 2.0, '2': 1.0}, {'V1': -1.0})

In [42]:
fptr.close()